In [1]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import seaborn as sns
from datetime import datetime
from tqdm.notebook import tqdm

from utils.neural_network import train_nn, get_device

from models.KF import KF, apply_kalman_filter
from utils.load_data import load_data, load_gt_data
from utils.preprocessing_v2 import Preprocessing, get_gt_diff_logs
import utils.results as results
from utils.results import bootstrap_ensemble, interactive_plot_predictions
import torch
import torch.nn as nn

TRENDS_FOLDER = 'data/google_trends/'
GDP_FOLDER = 'data/gdp/'
DATA_PREFIX = 'trends_data_by_topic_'

EPS = 1e-15
SEED = 42

plt.style.use('ieee.mplstyle')

## Loading and Preprocessing Data

### Loading Google Trends

In [2]:
TRAIN_PROPORTION = 0.9
MODE = "pct" # None | "pct" | "diff"
PERIOD = 4  # Year to year prediction

In [ ]:
data, all_gdps, all_gts = load_data()
data['country'].unique()

In [ ]:
preprocessor = Preprocessing(epsilon=EPS, gdp_diff_period=PERIOD, all_GDPs=all_gdps, all_GTs=all_gts)

X_train, y_train, X_valid, y_valid, x_hf = preprocessor.preprocess_data(train_pct=TRAIN_PROPORTION, 
                                                                  mode="diff", 
                                                                  take_log_diff_gdp=True,
                                                                  gt_trend_removal=False, 
                                                                  keep_pca_components=180, 
                                                                  noisy_data_stds=[0.005, 0.05, 0.1], 
                                                                  add_encoded_month=False, 
                                                                  gt_data_transformations=[get_gt_diff_logs], other_params={'plot_pca': False})

print(all_gts['country'].unique())

## Train 15 Prediction Models with different seeds

In [ ]:
N_MODELS = 15

models = []
for i in tqdm(range(N_MODELS)):
    model, training_loss, validation_loss, validation_r_squared, mse_losses = train_nn(X_train, 
                                                                        y_train, 
                                                                        X_valid, 
                                                                        y_valid, 
                                                                        num_epochs=100, 
                                                                        learning_rate=1e-4, 
                                                                        weight_decay=1e-2, 
                                                                        verbose=i == N_MODELS-1,
                                                                        seed=SEED+i)
    
    models.append(model)

del model, training_loss, validation_loss, validation_r_squared, mse_losses
ensembled_model = lambda x: torch.mean(torch.stack([model(x) for model in models]), dim=0)

In [6]:
device = get_device(False)

# Get the predictions
x_valid = torch.tensor(X_valid, dtype=torch.float32).to(device)
x_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_pred = models[0](x_valid).clone().detach().cpu().numpy().squeeze()
y_pred_train = models[0](x_train).clone().detach().cpu().numpy().squeeze()

# Associate the result by country and add 'Set' column
y_pred_country = pd.DataFrame({
    'date': preprocessor.dates_valid,
    'country': preprocessor.country_valid,
    'y_pred': y_pred,
    'y_true': y_valid,
    'Set': 'Validation'
})

y_pred_train_country = pd.DataFrame({
    'date': preprocessor.dates_train,
    'country': preprocessor.country_train,
    'y_pred': y_pred_train,
    'y_true': y_train,
    'Set': 'Training'
})

# Put together the train and the validation set
predictions = pd.concat([y_pred_train_country, y_pred_country])

# Melting the dataframe for better plotting
predictions_melted = predictions.melt(
    id_vars=["date", "country", "Set"],
    value_vars=["y_pred", "y_true"], 
    var_name="Type", value_name="Value"
)

In [ ]:
# Function to plot data for the selected country
def plot_by_country(selected_country, predictions_melted=predictions_melted):
    filtered_data = predictions_melted[predictions_melted["country"] == selected_country]

    plt.figure(figsize=(12, 6), dpi=300)

    sns.lineplot(
        data=filtered_data,
        x="date", y="Value", hue="Type", errorbar = None, style="Set", markers=True
    )
    
    plt.title(f"Prediction vs True Values for {selected_country}")
    plt.xlabel("Date")
    plt.ylabel("Values")
    plt.legend(title="Legend", loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Create a dropdown widget for selecting the country
countries = predictions["country"].unique()
dropdown = widgets.Dropdown(
    options=countries,
    value=countries[0],
    description='Country:'
)

# Use the interact function to link the dropdown with the plot function
interact(plot_by_country, selected_country=dropdown, predictions_melted=widgets.fixed(predictions_melted))

plt.show()


### Using the Kalman Filter on high frequency data

In [8]:
# Utility function to apply and plot the Kalman filter
def apply_and_plot_kalman_filter(model, preprocessor, use_true_values, accurate_noise_var, accel_var):
    kf_predictions_melted, hf_data_melted, kf_r2, kf_smoothness_loss = apply_kalman_filter(model, preprocessor, use_true_values=use_true_values, seed=SEED, accurate_noise_var=accurate_noise_var, accel_var=accel_var)

    all_predictions_merged = pd.concat([predictions_melted[predictions_melted['Type'] != 'y_pred'], kf_predictions_melted, hf_data_melted], ignore_index=True)

    # Create a dropdown widget for selecting the country
    countries = all_predictions_merged["country"].unique()
    dropdown = widgets.Dropdown(
        options=countries,
        value=countries[0],
        description='Country:'
    )

    mask_type = lambda df, type: df[df['Type'] == type]
    print(f"R2 for Kalman Filter: {kf_r2}, roughness: {kf_smoothness_loss[1]}")
    for col in ['y_pred', 'y_true']:
        means = results.measure_smoothness(mask_type(all_predictions_merged, col)['Value'].values, mask_type(all_predictions_merged, col)['date'], mask_type(all_predictions_merged, col)['country'])
        print(f"Geometric mean of roughness metrics on {col}: {means[1]}")

    # Use the interact function to link the dropdown with the plot function
    interact(plot_by_country, selected_country=dropdown, predictions_melted=widgets.fixed(all_predictions_merged))
    plt.show()

    # Return for paper plots
    return all_predictions_merged

In [ ]:
apply_and_plot_kalman_filter(ensembled_model, preprocessor, use_true_values=False, accurate_noise_var=0, accel_var=8e-6)

### Passing true values through the filter when available to correct the estimate

In [ ]:
apply_and_plot_kalman_filter(ensembled_model, preprocessor, use_true_values=True, accurate_noise_var=5e-4, accel_var=8e-6)

### Exploring noise variances

In [ ]:
variances = np.logspace(-12, 1, 100)

metrics = pd.DataFrame(index=pd.MultiIndex.from_product([variances, range(N_MODELS)], names=['accel_var', 'model']), columns=['r2', 'roughness'])                
ensemble_metrics = pd.DataFrame(columns=['r2', 'roughness'], index=pd.Index(variances, name='accel_var'))

for accel_var_idx, acceleration_noise_variance in tqdm(enumerate(variances), total=len(variances)):
    kf_predictions_melted, hf_data_melted, kf_r2, kf_smoothness_loss = apply_kalman_filter(ensembled_model, preprocessor, use_true_values=False, seed=SEED, accurate_noise_var=0, accel_var=acceleration_noise_variance)
    ensemble_metrics.loc[acceleration_noise_variance, 'r2'] = kf_r2
    ensemble_metrics.loc[acceleration_noise_variance, 'roughness'] = kf_smoothness_loss[1] # keep the geometric

    for model_idx, model in tqdm(enumerate(models), total=len(models), leave=False):
        kf_predictions_melted, hf_data_melted, kf_r2, kf_smoothness_loss = apply_kalman_filter(model, preprocessor, use_true_values=False, seed=SEED, accurate_noise_var=0, accel_var=acceleration_noise_variance)
        
        metrics.loc[(acceleration_noise_variance, model_idx), 'r2'] = kf_r2
        metrics.loc[(acceleration_noise_variance, model_idx), 'roughness'] = kf_smoothness_loss[1] # keep the geometric mean only

# Save the values for further use and plotting
metrics.to_csv('paper_data/kalman_filter_metrics.csv')
ensemble_metrics.to_csv('paper_data/kalman_filter_ensemble_metrics.csv')

# Take the median values
r2s = metrics.groupby('accel_var')['r2'].median().values
smoothness_losses = metrics.groupby('accel_var')['roughness'].median().values

optimal_metric = r2s - smoothness_losses

plt.figure(figsize=(8, 4))
plt.title(r"$R^2$ and roughness for different acceleration noise variances")
plt.plot(variances, r2s, label="$R^2$")
plt.plot(variances, ensemble_metrics['r2'], label="Ensemble $R^2$", color="purple")
plt.plot(variances, ensemble_metrics['roughness'], label="Ensemble roughness", color="orange")
plt.plot(variances, smoothness_losses, label="Roughness", color="green")
plt.plot(variances, optimal_metric, label="Optimal metric", color="red", linestyle="--")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("Acceleration noise variance")
plt.ylabel("Metric value")
plt.legend()
plt.show()

Finding the best value:

In [ ]:
OPTIMAL_ACCEL_VAR = variances[np.argmax(optimal_metric)]
print(f"Optimal acceleration noise variance: {OPTIMAL_ACCEL_VAR}")

In [ ]:
predictions_merged = apply_and_plot_kalman_filter(ensembled_model, preprocessor, use_true_values=False, accurate_noise_var=0, accel_var=OPTIMAL_ACCEL_VAR)

### Exploring noise variance values using true GDP values

In [ ]:
variances = np.logspace(-12, 2, 25)

r2s = np.zeros((len(variances), len(variances)))
smoothness_losses = np.zeros((len(variances), len(variances)))

metrics = pd.DataFrame(index=pd.MultiIndex.from_product([variances, variances, range(N_MODELS)], names=['accel_var', 'true_values_var', 'model']), columns=['r2', 'roughness'])                
ensemble_metrics = pd.DataFrame(columns=['r2', 'roughness'], index=pd.MultiIndex.from_product([variances, variances], names=['accel_var', 'true_values_var']))
for accel_var_idx, acceleration_noise_variance in tqdm(enumerate(variances), total=len(variances)):
    for true_values_var_idx, true_values_noise_variance in tqdm(enumerate(variances), total=len(variances), leave=False):
        kf_predictions_melted, hf_data_melted, kf_r2, kf_smoothness_loss = apply_kalman_filter(ensembled_model, preprocessor, use_true_values=True, seed=SEED, accurate_noise_var=true_values_noise_variance, accel_var=acceleration_noise_variance)
        ensemble_metrics.loc[(acceleration_noise_variance, true_values_noise_variance), 'r2'] = kf_r2
        ensemble_metrics.loc[(acceleration_noise_variance, true_values_noise_variance), 'roughness'] = kf_smoothness_loss[1] # keep the geometric

        for model_idx, model in tqdm(enumerate(models), total=len(models), leave=False):
            kf_predictions_melted, hf_data_melted, kf_r2, kf_smoothness_loss = apply_kalman_filter(model, preprocessor, use_true_values=True, seed=SEED, accurate_noise_var=true_values_noise_variance, accel_var=acceleration_noise_variance)
            
            metrics.loc[(acceleration_noise_variance, true_values_noise_variance, model_idx), 'r2'] = kf_r2
            metrics.loc[(acceleration_noise_variance, true_values_noise_variance, model_idx), 'roughness'] = kf_smoothness_loss[1] # keep the geometric mean only

# Save the values for further use and plotting
metrics.to_csv('paper_data/kalman_filter_metrics_with_true_values.csv')
ensemble_metrics.to_csv('paper_data/kalman_filter_ensemble_metrics_with_true_values.csv')

# take the medians across models
r2s = metrics.reset_index(drop=False).groupby(['accel_var', 'true_values_var'])['r2'].median().values.reshape((len(variances), len(variances))).astype(float)
smoothness_losses = metrics.reset_index(drop=False).groupby(['accel_var', 'true_values_var'])['roughness'].median().values.reshape((len(variances), len(variances))).astype(float)

# combination of both
optimal_metric = r2s - smoothness_losses

In [ ]:
def noise_variance_heatmap(r2s, smoothness_losses, optimal_metric, variances, file_path=None):
    fig, axes = plt.subplots(1, 3, figsize=(14, 4), sharey=True, sharex=True, dpi=300)
    fig.suptitle("Metrics for different noise variances")

    for ax in axes:
        ax.set_box_aspect(1)

    ticklabels = [f'{v:.2e}' if i % 2 == 0 else '' for i, v in enumerate(variances)] # show every second label

    # heatmap for R2
    sns.heatmap(r2s, annot=False, fmt=".2f", xticklabels=ticklabels, yticklabels=ticklabels, cmap="coolwarm", ax=axes[0])
    axes[0].set_title(r"$R^2$")
    axes[0].set_xlabel(r"var_\text{theory}")
    axes[0].set_ylabel(r"var_\text{true}")

    # heatmap for smoothness
    sns.heatmap(-smoothness_losses, annot=False, fmt=".2f", xticklabels=ticklabels, yticklabels=ticklabels, cmap="coolwarm", ax=axes[1])
    axes[1].set_title(r"$-e_s(\boldsymbol{\hat{y}}_h)$")
    axes[1].set_xlabel(r"$var_\text{theory}$")
    axes[1].set_ylabel(r"$var_\text{true}$")

    # heatmap for the optimal metric
    sns.heatmap(optimal_metric, annot=False, fmt=".2f", xticklabels=ticklabels, yticklabels=ticklabels, cmap="coolwarm", ax=axes[2])
    axes[2].set_title(r"$R^2 - e_s(\boldsymbol{\hat{y}}_h)$")
    axes[2].set_xlabel(r"$var_\text{theory}$")
    axes[2].set_ylabel(r"$var_\text{true}$")

    if file_path:
        plt.savefig(file_path, bbox_inches='tight')

    plt.tight_layout()
    plt.show()

noise_variance_heatmap(r2s, smoothness_losses, optimal_metric, variances)

We find the optimal parameters :

In [ ]:
optimal_indices = np.unravel_index(np.argmax(optimal_metric, axis=None), optimal_metric.shape)
OPTIMAL_ACCURATE_NOISE_VAR = variances[optimal_indices[1]]
OPTIMAL_ACCEL_VAR = variances[optimal_indices[0]]

print(f"Optimal acceleration noise variance: {OPTIMAL_ACCEL_VAR:.2e}")
print(f"Optimal true GDP noise variance: {OPTIMAL_ACCURATE_NOISE_VAR:.2e}")

In [ ]:
predictions_merged_true_value = apply_and_plot_kalman_filter(model, preprocessor, use_true_values=True, accurate_noise_var=OPTIMAL_ACCURATE_NOISE_VAR, accel_var=OPTIMAL_ACCEL_VAR)

### Paper plot

In [ ]:
SELECTED_COUNTRY = 'Germany'

# Prepare the data
type_mapping = {
    'y_pred': 'Neural network predictions',
    'y_kf': 'Filter predictions',
    'y_true': 'True values'
}

plot_data = predictions_merged.copy()
plot_data['Type'] = plot_data['Type'].apply(lambda x: type_mapping[x])
true_values = predictions_merged_true_value.loc[predictions_merged_true_value['Type'] == 'y_kf'].assign(Type='Filter predictions with true values')
plot_data = pd.concat([plot_data, true_values], ignore_index=True)

# Cutoff date
cutoff_date = (preprocessor.dates_valid.values[0] - preprocessor.dates_train.values[-1]) / 2 + preprocessor.dates_train.values[-1]

filtered_data = plot_data[(plot_data["country"] == SELECTED_COUNTRY) & (plot_data["Type"] != "Neural network predictions")]
high_freq_predictions = plot_data[(plot_data["country"] == SELECTED_COUNTRY) & (plot_data["Type"] == "Neural network predictions")]

plt.style.use('ieee.mplstyle')
plt.figure(figsize=(8, 5), dpi=300)

plt.plot(high_freq_predictions["date"], 
         high_freq_predictions["Value"],
         label="Neural network predictions",
         color="#1F77B4",
         linewidth=1,
         marker=',', 
         alpha=0.6)

sns.lineplot(
    data=filtered_data,
    x="date", y="Value", hue="Type", errorbar = None, style="Set", markers=True, markersize=3.8
)
plt.title(f"Kalman filter prediction vs true values for {SELECTED_COUNTRY}")
plt.xlabel("Date")
plt.ylabel("Normalised prediction")
plt.axvline(cutoff_date, color='red', linestyle='--', label='Start of Validation Set', linewidth=1)
plt.legend(
    loc="upper center",  # Position the legend in the center
    bbox_to_anchor=(0.5, -0.15),  # Center it below the plot
    ncol=3,  # Arrange items in a single row
    frameon=False  # Optional: Remove the legend frame
)
plt.grid(True)

plt.savefig(results.OUTPUT_DATA_PATH + f'KF_{SELECTED_COUNTRY.replace(' ', '_')}.pdf', bbox_inches='tight')

plt.tight_layout()
plt.show()

In [ ]:
plt.style.use('ieee.mplstyle')
noise_variance_heatmap(r2s, smoothness_losses, optimal_metric, variances, file_path=results.OUTPUT_DATA_PATH + 'KF_heatmap.pdf')

In [ ]:
plt.style.use('ieee.mplstyle')

metrics = pd.read_csv('paper_data/kalman_filter_metrics.csv', index_col=[0, 1])
ensemble_metrics = pd.read_csv('paper_data/kalman_filter_ensemble_metrics.csv', index_col=0)
metrics_true_values = pd.read_csv('paper_data/kalman_filter_metrics_with_true_values.csv', index_col=[0, 1, 2])
ensemble_metrics_true_values = pd.read_csv('paper_data/kalman_filter_ensemble_metrics_with_true_values.csv', index_col=[0, 1])

c = metrics['r2'].values - metrics['roughness'].values
ce = ensemble_metrics['r2'].values - ensemble_metrics['roughness'].values
c_tv = metrics_true_values['r2'].values - metrics_true_values['roughness'].values
ce_tv = ensemble_metrics_true_values['r2'].values - ensemble_metrics_true_values['roughness'].values

# Create the figure
plt.figure(figsize=(8, 4), dpi=150)

# Scatter plots

sc_tv = plt.scatter(metrics_true_values['roughness'].values, metrics_true_values['r2'].values,
                    c=c_tv, cmap='Blues', marker='o', s=20, alpha=0.7, label='Batch Size 6')

sc = plt.scatter(metrics['roughness'].values, metrics['r2'].values,
                    c=c, cmap='Reds', marker='o', s=20, alpha=0.7, label='Batch Size 4')

sce = plt.scatter(ensemble_metrics['roughness'].values, ensemble_metrics['r2'].values,
                    c=ce, cmap='Reds', marker='x', s=20, alpha=0.7, label='Batch Size 4')

sce_tv = plt.scatter(ensemble_metrics_true_values['roughness'].values, ensemble_metrics_true_values['r2'].values,
                    c=ce_tv, cmap='Blues', marker='x', s=20, alpha=0.7, label='Batch Size 6')

plt.scatter(0.6549, 0.1512, label='Median Reference Model', color='green', marker='o', s=25)
plt.scatter(0.4968, 0.4515, label='Ensembling on Reference Model', color='green', marker='x', s=25)

# Add labels, title, and grid
plt.xlabel(r'Smoothness Loss $e_s(\boldsymbol{\hat{y}_h})$')
plt.ylabel('Validation $R^2$')
plt.title(r'Validation $R^2$ vs. High-Frequency Smoothness Loss $e_s(\boldsymbol{\hat{y}_h})$')

# Custom legend with color markers for maximum c values
plt.legend(handles=[
    plt.Line2D([0], [0], marker='s', color='w', label='Kalman Filter on reference model',
               markerfacecolor='red', markeredgecolor='red', markersize=7),
    plt.Line2D([0], [0], marker='s', color='w', label=r'Kalman Filter on reference model with true values',
               markerfacecolor='blue', markeredgecolor='blue', markersize=7),
    plt.Line2D([0], [0], marker='s', color='w', label='Reference Model',
               markerfacecolor='green', markeredgecolor='green', markersize=7),
               
plt.Line2D([0], [0], marker='x', color='w', label='Ensembled Predictions',
               markerfacecolor='black', markeredgecolor='black', markersize=7),
    plt.Line2D([0], [0], marker='o', color='w', label='Median Model',
               markerfacecolor='black', markeredgecolor='black', markersize=7),
], loc='center', bbox_to_anchor=(0.5, -0.3), ncol=2, frameon=False),

#plt.ylim(0, 0.55)
#plt.xlim(0, 0.7)
plt.grid()

plt.savefig('data/output_for_paper/kalman_filter_scatter.pdf', bbox_inches='tight')

# Show the plot
plt.show()
